# Main-and-Transfer Substation Test

## Create a New Substation

### Import CIMantic Graphs Library and CIM profile

In [1]:
# Import transmission and distribution modeling classes
from cimgraph.models import FeederModel, NodeBreakerModel
from cimgraph.databases import ConnectionParameters, RDFlibConnection, BlazegraphConnection
import cimgraph.utils as utils

import importlib

cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

params = ConnectionParameters(filename=None, cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)

### Create New Main-and-Transfer Substation

In [2]:
from cimbuilder.substation_builder import MainAndTransferSubstation

In [3]:
SubBuilder = MainAndTransferSubstation(connection=connection, name="main_transfer_sub", base_voltage = 11500)
substation = SubBuilder.substation

### Add IEEE 13 Feeder and IEEE 13 Assets Feeder to substation

In [4]:
# Import 13 bus model from XML file
ieee13_feeder = cim.Feeder(mRID = '49AD8E07-3BF9-A4E2-CB8F-C3722F837B62')
params = ConnectionParameters(filename='../test_models/IEEE13.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
ieee13_network = FeederModel(connection=connection, container=ieee13_feeder, distributed=False)


In [5]:

assets13_feeder = cim.Feeder(mRID = '5B816B93-7A5F-B64C-8460-47C17D6E4B0F')
params = ConnectionParameters(filename='../test_models/IEEE13_Assets.xml', cim_profile=cim_profile, iec61970_301=8)
connection = RDFlibConnection(params)
assets13_network = FeederModel(connection=connection, container=assets13_feeder, distributed=False)


In [6]:
SubBuilder.new_main_trans_feeder(series_number = 10, feeder=ieee13_feeder, feeder_network=ieee13_network)
SubBuilder.new_main_trans_feeder(series_number = 20, feeder=assets13_feeder, feeder_network=assets13_network)

In [7]:
SubBuilder.network.pprint(cim.Substation)
SubBuilder.network.pprint(cim.Feeder)

{
    "f6b9d448-7b02-43a0-91e6-bf83cda64c56": {
        "mRID": "f6b9d448-7b02-43a0-91e6-bf83cda64c56",
        "name": "main_transfer_sub"
    }
}
{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['673E896A-DCBF-4E43-9924-BEB31C5B6005', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', '421E99BE-A834-4809-B924-84D88F634A45', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '0124E881-B82D-4206-BBDF-37D585159872', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', '0A98A62D-7642-4F03-8317-A8605CBDBA37', '04984C4D-CC29-477A-9AF4-61AC7D74F16F', '0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', 'DC889FA5-7B28-4273-A1D7

### Save Substation to XML File

In [8]:
utils.write_xml(SubBuilder.network, '../test_output/main_and_transfer.xml')

## Round-Trip Test: Load and Read from Database

### Delete all old entries from database and load new models

In [9]:
# Connect to Blazegraph Database
from cimgraph.databases import BlazegraphConnection
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
blazegraph = BlazegraphConnection(params)
blazegraph.execute('drop all')

/home/ande188/CIM-Builder/.venv/lib/python3.10/site-packages/SPARQLWrapper/Wrapper.py:1215: RuntimeWarning: unknown response content type 'text/html;charset=utf-8' returning raw response...
  warnings.warn(


b'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd"><html><head><meta http-equiv="Content-Type" content="text&#47;html;charset=UTF-8"><title>blazegraph&trade; by SYSTAP</title\n></head\n><body<p>totalElapsed=20ms, elapsed=20ms, connFlush=0ms, batchResolve=0, whereClause=0ms, deleteClause=0ms, insertClause=0ms</p\n><hr><p>COMMIT: totalElapsed=35ms, commitTime=1707879336481, mutationCount=6671</p\n></html\n>'

In [10]:
from cimloader.databases.blazegraph import BlazegraphConnection as BlazegraphLoader
params = ConnectionParameters(url = "http://localhost:8889/bigdata/namespace/kb/sparql", cim_profile=cim_profile, iec61970_301=8)
loader = BlazegraphLoader(params)

In [11]:
loader.upload_from_file(filename='../test_models/IEEE13.xml')
loader.upload_from_file(filename='../test_models/IEEE13_Assets.xml')
loader.upload_from_file(filename='../test_output/main_and_transfer.xml')

HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="3392" milliseconds="31"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 62
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="2927" milliseconds="30"/>HTTP/1.1 100 Continue

HTTP/1.1 200 OK
Content-Type: application/xml;charset=iso-8859-1
Content-Length: 61
Server: Jetty(9.4.18.v20190429)

<?xml version="1.0"?><data modified="282" milliseconds="15"/>

### Create new NodeBreakerModel of Substation + Feeders

In [12]:
network = NodeBreakerModel(container=substation, connection=blazegraph, distributed = False)

In [13]:
# Print substation info
network.get_all_edges(cim.Substation)
network.pprint(cim.Substation)

{
    "f6b9d448-7b02-43a0-91e6-bf83cda64c56": {
        "mRID": "f6b9d448-7b02-43a0-91e6-bf83cda64c56",
        "name": "main_transfer_sub",
        "ConnectivityNodes": "['94443d21-e85e-4173-a88e-ba7a3efaebc8', 'bda6fb7e-dd5b-4152-b28e-39c3ad9f9ed1', 'c77dee68-e288-4a49-97a4-94e33fbb2a20', 'd24f2202-d426-465f-84e8-96182c9be202', 'f03b9ff5-f4e7-4a44-bbc7-9026558bcb2d', '68160a60-8614-40ef-8832-c9923b2e69c7']",
        "Equipments": "['8234de63-5b09-47de-afd0-b75876469820', '9561c211-e340-4b5d-9a47-ed435b1ff0ee', '9d32aa41-4451-458a-9818-b6c830b4e2f7', 'bb17b4c2-d17b-4861-809d-bdc31619185d', 'c061bd3b-ce62-4b58-905a-a10b9c8604c7', 'e541844f-41b6-4bd2-b138-cf5d2c42c81f', 'e7de501d-8f29-478f-9613-dca16cd9fe7f', 'ee23fd98-c6ae-4ea0-97a3-50804c0955c0', 'ee7f94aa-d0cd-4c4b-b661-66b558e09179', '0453c689-ca50-44b0-82a9-47ff0f64bc54', '13a4de9b-00b3-41c4-bbde-0c775c3e0ea4', '37e48b0d-f6f9-4974-8b61-05e339f606c9', '6d8b2c22-8ae1-47e6-b66c-ee414304950b']",
        "NormalEnergizedFeeder": "['49AD

In [14]:
# Print feeder info
network.get_all_edges(cim.Feeder)
network.pprint(cim.Feeder)

{
    "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62": {
        "mRID": "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62",
        "name": "ieee13nodeckt",
        "Location": "8E4E3C92-0B7A-4F74-8FD2-CC10F74E452F",
        "ConnectivityNodes": "['0DCC57AF-F4FA-457D-BB24-2EFDA9865A1A', '0F1E28C3-6C44-4F88-B79C-2FDBCA4487B2', '2A6DC4DD-D3DC-434D-A187-D2C58A0A72C8', '30BE5988-DE57-4E0C-AB08-50D5A13D2C1B', '421E99BE-A834-4809-B924-84D88F634A45', '63DFBEA0-CD06-4D2E-B956-DF9517BE057B', '673E896A-DCBF-4E43-9924-BEB31C5B6005', '6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B', '76D6D03C-967C-4E12-836D-A98DF3552BC7', '7BEDDADD-0A14-429F-8601-9EA8B892CA6E', '8C58660F-C62C-4903-BE72-22F1255B1E62', '8E99F99D-FE8F-420B-AC49-0B52DF5362AB', '94F822E0-7130-4205-8597-B47110BBEF4B', 'A8A25B50-3AE3-4A31-A18B-B3FA13397ED3', 'ADDB7A30-5A3C-4179-AF5D-5C9A7213B0E7', 'C6256170-E6ED-4F91-8EBD-748090C3FDD5', 'DBFA8EFA-B397-40DA-9642-60E8DE4B3471', 'DC889FA5-7B28-4273-A1D7-205BE3E0BFED', 'E5B2888B-B60D-4DA6-A4F7-17EB849D28B2', '0124E881-B8

In [15]:
# Print total load served by substation from both feeders
total_load = 0
network.get_all_edges(cim.EnergyConsumer)
for load in network.graph[cim.EnergyConsumer].values():
    total_load = total_load + float(load.p)

print(f'total load is {total_load/1000} kW')

total load is 6937.0 kW


### Merge substation and feeders into a single XML file 

In [16]:
utils.get_all_data(network)

In [17]:
utils.write_xml(network, '../test_output/main_transfer_and_feeders.xml')
